This notebook contains the code used to generate embeddings  
  
The Higashi Wiki is used as source  
    https://github.com/ma-compbio/Higashi/wiki/Fast-Higashi-Usage

In [1]:
import pandas as pd
import pickle

/tmp/ipykernel_5420/3842437144.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from fasthigashi.FastHigashi_Wrapper import *

In [3]:
# peek at the data, make sure looks okay
data = pd.read_csv(
    "data.txt", 
    sep="\t", nrows=5)
data

,cell_id,chrom1,pos1,chrom2,pos2,count
0,1,chr1,100000,chr7,158600000,1
1,1,chr1,700000,chr10,39100000,1
2,1,chr1,900000,chr5,151500000,1
3,1,chr1,950000,chr1,66950000,1
4,1,chr1,950000,chr15,83600000,1


In [4]:
import os
os.listdir()

['data.txt',
 'config_pfc.JSON',
 'GenerateEmbeddings.ipynb',
 '.ipynb_checkpoints',
 'config.json',
 'label_info.pickle',
 'cytoBand_hg19.txt',
 'hg19.chrom.sizes.txt',
 'temp']

In [6]:
# initialize the model
model1 = FastHigashi(config_path="config_pfc.JSON", 
                     path2input_cache=None,      
                     path2result_dir=None,
                     off_diag=100,  # maximum number of diagonals to consider
                     filter=True,   # only use cells that pass QC to learn meta interactions?
                     do_conv=True,  # use linear convolution?
                     do_rwr=True,   # use partial random walk with restart?
                     do_col=False,  # use sqrt_vc normalization? Will use automatically if needed
                     no_col=False)  # force to use sqrt_vs normalization?

# process the data from contacts to sparse matrix
model1.fast_process_data()

# pack from sparse matrix to sparse tensors
model1.prep_dataset(batch_norm=True) # do batch normalization?

setting to gpu:0 available memory = 6141 MB
generating start/end dict for chromosome
extracting from data.txt
First calculating how many lines are there
There are 1330348175 lines


 - Processing :   0%|          | 0/1330348175 [00:00<?, ?it/s]

fast process finishes
total number of cells that pass qc check 4145 bad 93 total: 4238
cache file = temp/cache_intra_500000_offdiag_100_.pkl
will do per batch normalization
saving cached input to temp/cache_intra_500000_offdiag_100_.pkl


sparse mtx into tensors:   0%|          | 0/22 [00:00<?, ?it/s]

breaking into batches:   0%|          | 0/22 [00:00<?, ?it/s]

sparsity 0.05076833148571178
do_conv True do_rwr True do_col False
recommend_bs_cell [2073, 2073, 4145, 2073, 2073, 2073, 2073, 4145, 2073, 4145, 4145, 4145, 4145, 4145, 4145, 4145, 4145, 4145, 2073, 2073, 4145, 4145] pinning memory


In [8]:
# run the model
model1.run_model(dim1=.6, rank=256, n_iter_parafac=1, extra="")

dim1_0.6_rank_256_niterp_1_
n_iter_parafac 1
empty params initialized
time elapsed: 1.05


initializing params:   0%|          | 0/22 [00:00<?, ?it/s]

rwr iters: [5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 4 5 5 5 4 4]
time elapsed: 299.64
finish init
Starting iteration 0

PARAFAC2 re=7.005 takes 46.9s
Starting iteration 1

PARAFAC2 re=0.325 9.98e-01 variation min9.9e-01 at chrom 21, max1.0e+00 at chrom 1 takes 45.4s
Starting iteration 2

PARAFAC2 re=0.309 9.54e-02 variation min5.3e-02 at chrom 13, max1.2e-01 at chrom 20 takes 47.0s
Starting iteration 3

PARAFAC2 re=0.300 5.80e-02 variation min5.2e-02 at chrom 18, max8.3e-02 at chrom 15 takes 39.9s
Starting iteration 4

PARAFAC2 re=0.298 1.62e-02 variation min1.3e-02 at chrom 18, max2.9e-02 at chrom 14 takes 39.0s
Starting iteration 5

PARAFAC2 re=0.297 6.32e-03 variation min3.8e-03 at chrom 19, max1.2e-02 at chrom 10 takes 38.4s
Starting iteration 6

PARAFAC2 re=0.296 3.19e-03 variation min1.2e-03 at chrom 19, max6.1e-03 at chrom 13 takes 38.9s
Starting iteration 7

PARAFAC2 re=0.296 1.88e-03 variation min6.8e-04 at chrom 19, max3.6e-03 at chrom 13 takes 38.9s
Starting iteration 8

PARAFAC2 re

In [10]:
# get the results
embedding256 = model1.fetch_cell_embedding(final_dim=256, restore_order=True)
embedding128 = model1.fetch_cell_embedding(final_dim=128, restore_order=True)
embedding64 = model1.fetch_cell_embedding(final_dim=64, restore_order=True)

fetching embedding
fetching embedding
fetching embedding


In [13]:
# dict keys that are available:
# 'embed_raw' = raw embeddings
# 'embed_l2_norm' = embeddings after L2 normalization
# 'embed_correct_coverage_fh' = embeddings after linear correction of coverage
# 'embed_l2_norm_correct_coverage_fh' = embeddings after linear correction of coverage + L2 normazliation

# authors suggest:
# It is suggested to try 'embed_l2_norm' or 'embed_l2_norm_correct_coverage_fh'. 
# Although technically 'embed_l2_norm_correct_coverage_fh' offers embeddings that 
# are unbiased towards sequencing depths of cells, for some datasets, the cell type 
# information is highly correlated with the sequencing depths. A simple regression 
# would lead to much worse embedding results. This is caused by the non-orthogonality 
# of biological variance and technical variance, and usually happens when two scHi-C 
# libraries contain different (or even non-overlapping) cell types but also have 
# different sequencing depths. Thus, it is suggested to test out both embeddings and 
# see which gives more reasonable results.

In [12]:
with open('embedding256.pickle', 'wb') as fp:
    pickle.dump(embedding256, fp)
    print('dictionary saved successfully to file')
    
with open('embedding128.pickle', 'wb') as fp:
    pickle.dump(embedding128, fp)
    print('dictionary saved successfully to file')
        
with open('embedding64.pickle', 'wb') as fp:
    pickle.dump(embedding64, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file
dictionary saved successfully to file
